# Pytorch tutorial 1 
this is a tutorial on how to use pytorch building a basic neural network 

First we have to import the libaries that are needed 

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

Now that we have the necessary libaries imported we can now  import out test and training data from the MNIST data set 

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Now that we have the test and training data loaded we can now pass the dataset to dataloader. This will wrap an iterable over our dataset, and supporst batching, sampling, shuffling and multiprocess data loading. We are going to define a batch size of 64. This means that each element in the dataloader iterable will return a batch of 64 features and labels 

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


Now we can define a nural network, we do this by using nn.Module. We are going to define two functions a init function and a forward function.  We define the layers of the network in the __ in it __ function. And we specify how data will pass through the network in the forward function. The speed things up we will use a GPU if available if not we will use CPU

In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


We know that to train a neural network we need two things 
- 1) loss fucntion
- 2) optimizer  

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

 Now we are going to define a function for training our model and backpropagating the error to adjust the hyperparameters 

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We now are going to check the models perfromacne againgst the test dataset. This will tell us that the model is in fact learning. 


In [7]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Training is conducted over multiple iterations called epochs. During each epoch, the model learns paraemters to make better predictions. We will print the models accuracy and loss at each epoch. We should see the accuracy incrase and the loss decrease through each epoch     

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306691  [    0/60000]
loss: 2.301905  [ 6400/60000]
loss: 2.297332  [12800/60000]
loss: 2.287883  [19200/60000]
loss: 2.276832  [25600/60000]
loss: 2.274376  [32000/60000]
loss: 2.268922  [38400/60000]
loss: 2.270203  [44800/60000]
loss: 2.251479  [51200/60000]
loss: 2.225602  [57600/60000]
Test Error: 
 Accuracy: 38.6%, Avg loss: 0.035184 

Epoch 2
-------------------------------
loss: 2.254609  [    0/60000]
loss: 2.259766  [ 6400/60000]
loss: 2.246105  [12800/60000]
loss: 2.223297  [19200/60000]
loss: 2.207486  [25600/60000]
loss: 2.198854  [32000/60000]
loss: 2.183946  [38400/60000]
loss: 2.182044  [44800/60000]
loss: 2.152265  [51200/60000]
loss: 2.111481  [57600/60000]
Test Error: 
 Accuracy: 44.6%, Avg loss: 0.033513 

Epoch 3
-------------------------------
loss: 2.168022  [    0/60000]
loss: 2.174188  [ 6400/60000]
loss: 2.131421  [12800/60000]
loss: 2.096326  [19200/60000]
loss: 2.094464  [25600/60000]
loss: 2.058376  [32000/600

Now we are going to save the model

In [9]:
torch.save(model.state_dict(),'model.pth')
print('Saved Pytorch model state to model.pth')

Saved Pytorch model state to model.pth


 The process for loading a model includes re creating the model structure and loading the data dictionary into it 
 

In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [11]:
classes = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot',
    
]
model.eval()
x,y=test_data[0][0],test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted,actual = classes[pred[0].argmax(0)],classes[y]
    print(f'Predicted: "{predicted}",Actual: "{actual}"')

Predicted: "Sneaker",Actual: "Ankle boot"


# References 

https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html